# Applied Process Mining Module

This notebook is part of an Applied Process Mining module. The collection of notebooks is a *living document* and subject to change. 

# Assignment - BPI Challenge 2020

## Setup

<img src="https://pm4py.fit.fraunhofer.de/static/assets/images/pm4py-site-logo-padded.png" alt="PM4Py" style="width: 200px;"/>

In this notebook, we are using the [PM4Py library](https://pm4py.fit.fraunhofer.de/) in combination with several standard Python data science libraries:

* [pandas](https://pandas.pydata.org/)
* [plotnine](https://plotnine.readthedocs.io/en/stable/)

In [1]:
## Perform the commented out commands to install the dependencies
# %pip install pandas
# %pip install matplotlib
# %pip install pm4py

In [2]:
import pandas as pd
import pm4py
import plotnine
from plotnine import ggplot, geom_point, aes, theme_bw, coord_flip, scale_y_discrete, theme, element_text, geom_bin2d

You may use this notebook to conduct the analysis.

## Dataset

The proposed real-life dataset to investigate is the *BPI Challenge 2020* dataset. The dataset is captured from the travel reimbursment process of Eindhoven University of Technolog and has been collected for usage in the BPI challenge. The BPI challenge is a yearly event in the Process Mining research community in which an event log is released along with some business questions that shall be addressed with process analytics techniques.

Here is more informaation on the dataset and downloads links to the data files:

* [Overview of the Case](https://icpmconference.org/2020/bpi-challenge/)
* [Dataset](https://doi.org/10.4121/uuid:52fb97d4-4588-43c9-9d04-3604d4613b51)

On the BPI Challenge 2020 website above, there are several reports (including the winners of the challenge) that describe and analyze the dataset in detail. However, we suggest that you first try to explore the dataset without reading the reports. The business questions and a description of the process flow can be also found at the BPI Challenge 2020 website. We repeat it here for convenience:

### Process Flow

The various declaration documents (domestic and international declarations, pre-paid travel costs and requests for payment) all follow a similar process flow. After submission by the employee, the request is sent for approval to the travel administration. If approved, the request is then forwarded to the budget owner and after that to the supervisor. If the budget owner and supervisor are the same person, then only one of the these steps it taken. In some cases, the director also needs to approve the request.

In all cases, a rejection leads to one of two outcomes. Either the employee resubmits the request, or the employee also rejects the request.

If the approval flow has a positive result, the payment is requested and made.

The travel permits follow a slightly different flow as there is no payment involved. Instead, after all approval steps a trip can take place, indicated with an estimated start and end date. These dates are not exact travel dates, but rather estimated by the employee when the permit request is submitted. The actual travel dates are not recorded in the data, but should be close to the given dates in most cases.

After the end of a trip, an employee receives several reminders to submit a travel declaration.

After a travel permit is approved, but before the trip starts, employees can ask for a reimbursement of pre-paid travel costs. Several requests can be submitted independently of each other. After the trip ends, an international declaration can be submitted, although sometimes multiple declarations are seen for specific cases.

It’s important to realize that the process described above is the process for 2018. For 2017, there are some differences as this was a pilot year and the process changed slightly on several occasions.

### Business Questions

The following questions are of interest:

* What is the throughput of a travel declaration from submission (or closing) to paying?
* Is there are difference in throughput between national and international trips?
* Are there differences between clusters of declarations, for example between cost centers/departments/projects etc.?
* What is the throughput in each of the process steps, i.e. the submission, judgement by various responsible roles and payment?
* Where are the bottlenecks in the process of a travel declaration?
* Where are the bottlenecks in the process of a travel permit (note that there can be mulitple requests for payment and declarations per permit)?
* How many travel declarations get rejected in the various processing steps and how many are never approved?

Then there are more detailed questions

* How many travel declarations are booked on projects?
* How many corrections have been made for declarations?
* Are there any double payments?
* Are there declarations that were not preceded properly by an approved travel permit? Or are there even declarations for which no permit exists?
* How many travel declarations are submitted by the traveler and how many by a mandated person?
* How many travel declarations are first rejected because they are submitted more than 2 months after the end of a trip and are then re-submitted?
* Is this different between departments?
* How many travel declarations are not approved by budget holders in time (7 days) and are then automatically rerouted to supervisors?
* Next to travel declarations, there are also requests for payments. These are specific for non-TU/e employees. Are there any TU/e employees that submitted a request for payment instead of a travel declaration?

Similar to the task at the BPI challenge, we are aware that not all questions can be answered on this dataset and we encourage you to come up with new and interesting insights.

## Data Loading

To simplify the data loading task, here are the initial steps:

In [5]:
import requests
import shutil

def download_file(url, local_filename):
    with requests.get(url, stream=True) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

    return local_filename

rfp_data_file = download_file("https://data.4tu.nl/ndownloader/files/24061154", "../data/rfp.xes.gz")
ptc_data_file = download_file("https://data.4tu.nl/ndownloader/files/24043835", "../data/ptc.xes.gz")
int_decl_data_file = download_file("https://data.4tu.nl/ndownloader/files/24023492", "../data/int_decl.xes.gz")
dom_decl_data_file = download_file("https://data.4tu.nl/ndownloader/files/24031811", "../data/dom_decl.xes.gz")

Next we are going to load the XES file into memory using PM4Py.

In [7]:
import pm4py

rfp_data = pm4py.read_xes(rfp_data_file)
ptc_data = pm4py.read_xes(ptc_data_file)
int_decl_data = pm4py.read_xes(int_decl_data_file)
dom_decl_data = pm4py.read_xes(dom_decl_data_file)

parsing log, completed traces :: 100%|██████████| 10500/10500 [00:02<00:00, 4410.17it/s]
